# BTCUSDT-ML-ETL-Pipeline

## Download Repository

In [ ]:
!git clone https://github.com/danhtran2mind/BTCUSDT-ML-ETL-Pipeline.git
%cd BTCUSDT-ML-ETL-Pipeline

Cloning into 'BTC-USDT-ETL-Pipeline'...
remote: Enumerating objects: 629, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 629 (delta 177), reused 169 (delta 92), pack-reused 367 (from 1)
Receiving objects: 100% (629/629), 413.36 KiB | 17.97 MiB/s, done.
Resolving deltas: 100% (358/358), done.
/content/BTC-USDT-ETL-Pipeline


## Install Dependencies

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 107.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895798 sha256=8d5023a79050532ecc7bd904ed3c6d9c31b743bd59acfb8c6308770400192716
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


## Setup Modules

### MinIO

In [ ]:
!python scripts/install_minio.py

--2025-10-28 17:17:56--  https://dl.min.io/server/minio/release/linux-amd64/minio
Resolving dl.min.io (dl.min.io)... 178.128.69.202, 138.68.11.125
Connecting to dl.min.io (dl.min.io)|178.128.69.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110989496 (106M) [application/octet-stream]
Saving to: ‘minio’

minio               100%[===================>] 105.85M  14.7MB/s    in 45s     

2025-10-28 17:18:42 (2.34 MB/s) - ‘minio’ saved [110989496/110989496]



In [ ]:
%%writefile minio.env
MINIO_ROOT_USER=your_username
MINIO_ROOT_PASSWORD=your_password
MINIO_HOST=localhost:9192
MINIO_CONSOLE_ADDRESS=localhost:9193

Writing minio.env


In [ ]:
%%bash
export MINIO_ROOT_USER=your_username
export MINIO_ROOT_PASSWORD=your_password
# Start server with credentials
MINIO_ROOT_USER=$MINIO_ROOT_USER MINIO_ROOT_PASSWORD=$MINIO_ROOT_PASSWORD \
./minio server ~/minio-data --address :9192 --console-address :9193 > logs/minio_server.log 2>&1 &

### Sparks

In [ ]:
%%bash
python scripts/install_spark.py

Extracting archive...
Spark installed to: /root/spark
Environment variables set for this session!

To verify installation:
!ls $SPARK_HOME
!spark-submit --version


/content/BTC-USDT-ETL-Pipeline/scripts/install_spark.py:37: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=".")


### Airflow

In [ ]:
%%bash
python scripts/install_airflow.py

Setup at /content/BTC-USDT-ETL-Pipeline/airflow, Login: admin/admin at http://localhost:8081


## Verify Ports in Use

In [ ]:
!netstat -tuln | grep 8081

tcp        0      0 0.0.0.0:8081            0.0.0.0:*               LISTEN     


In [ ]:
!netstat -tuln | grep 9192

tcp        0      0 127.0.0.1:9192          0.0.0.0:*               LISTEN     
tcp6       0      0 :::9192                 :::*                    LISTEN     


## Use Zrok Tunneling Service

In [ ]:
!wget https://github.com/openziti/zrok/releases/download/v1.1.10/zrok_1.1.10_linux_amd64.tar.gz
!tar -xf zrok_1.1.10_linux_amd64.tar.gz

--2025-10-28 17:20:05--  https://github.com/openziti/zrok/releases/download/v1.1.10/zrok_1.1.10_linux_amd64.tar.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/515311500/c675ed4b-a6cf-482a-b3e7-4adcfd283837?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-28T18%3A18%3A59Z&rscd=attachment%3B+filename%3Dzrok_1.1.10_linux_amd64.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-28T17%3A18%3A19Z&ske=2025-10-28T18%3A18%3A59Z&sks=b&skv=2018-11-09&sig=e4q5Iim5LCwy%2BiXrTl0qv64GMkr8aLg%2F20sS6YkUJWc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MTY3MzgwNSwibmJmIjoxNzYxNjcyMDA1LCJwYXRoIjoic

In [ ]:
!chmod +x zrok

In [ ]:
import os
import subprocess
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("zrok_tunneling.env")

# Retrieve the token from the environment
zrok_token = os.getenv('ZROK_TOKEN')
if not zrok_token:
    raise ValueError("ZROK_TOKEN not found in .env file. Please check your .env file.")

# Path to the zrok binary (adjust if needed)
zrok_path = './zrok'  # Use './zrok.exe' on Windows

# Run the zrok enable command
try:
    result = subprocess.run([zrok_path, 'enable', zrok_token],
                           capture_output=True, text=True, check=True)
    print("Command output:")
    print(result.stdout)
    if result.stderr:
        print("Warnings/Errors:")
        print(result.stderr)
except subprocess.CalledProcessError as e:
    print(f"Error running zrok enable: {e}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
except FileNotFoundError:
    print(f"zrok binary not found at {zrok_path}. Ensure it's in the current directory and executable.")

Command output:
open /dev/tty: no such device or address



In [ ]:
!./zrok share public 8081 --headless